# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.2.0](https://python.langchain.com/v0.2/docs/versions/v0_2/)
  4. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas) framework.
  

- 🤝 Breakout Room Part #2:
  1. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage `GPT-3.5-Turbo` as the `critic_llm`!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [ ]:
! pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.6 MB/s eta 0:00:00


We'll also get the "star of the show" today, which is Ragas!

In [1]:
! pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [2]:
#! pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [3]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [4]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

documents = loader.load()

In [5]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [7]:
len(documents)

1864

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [9]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="PMarca Blogs",
)

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!

**Answer**


HNSW Graphs: These are used for efficient indexing. They allow Qdrant to sift through high-dimensional data quickly, reducing the time it takes to find relevant results.

Product Quantization: This is a technique used to compress vectors. It ensures that the engine is not just fast but also memory-efficient.

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [10]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [11]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [12]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change\nhave a reasonable chance at succeeding.\nSecond rule of thumb:\nOnce you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '2ffd8ea39ce24a139c8c22c39641095a', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:\nIn a rapidly changing Held like technology, the best place to\nget experience when you’re starting out is in younger, high-\ngrowth companies.' metadata={'source': 'https://d1lamhf6l6yk6d

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [15]:
! pip install langchainhub

In [16]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [17]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [18]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [19]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

**Answer**

* we are creating llm, prompt template and retriver
* then we wrap everything in qa chain
* in first step we provide question and retriver
* output of that is context and question key
* at the end we are just collecting context from previous step for context key output
* and response is generated by chaining prompt template (collect context and question data from provided keys)  with llm

Let's test it out!

In [20]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Get right to the center of the industry.


In [21]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.', metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': 'c6b24d6b32c4451089069dcab9304d1d', '_collection_name': 'PMarca Blogs'}), Document(page_content='watching carefully — if everyone agrees right up front that\nwhatever you are doing makes total sense, it probably isn’t a new\nand radical enough idea to justify a

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [22]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

several reasons:

Rag evaluation requires a diverse set of questions and scenarios to assess the model's generation quality comprehensively, and for that we need bigger chunks!


In [23]:
len(eval_documents)

624


> NOTE: 🛑 Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage GPT-3.5-Turbo as the critic_llm. If you're attempting to create a lot of samples please be aware of cost, as well as rate limits. 🛑

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo") # <--- If you don't have GPT-4 access, or to reduce cost/rate limiting issues.
# critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
testset.to_pandas()

####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).


**Answer**

Reasoning: Rewrite the question in a way that enhances the need for reasoning to answer it effectively.

Conditioning: Modify the question to introduce a conditional element, which adds complexity to the question.

Multi-Context: Rephrase the question in a manner that necessitates information from multiple related sections or chunks to formulate an answer.

Let's look at the output and see what we can learn about it!

In [26]:
testset.test_data[0]

DataRow(question='Why does the author believe that intelligence is overrated?', contexts=['Criteria 7rst\nLots of people will tell you to hire for intelligence.\nEspecially in this industry.\nYou will read, hire the smartest people out there and your com-\npany’s success is all but guaranteed.\nI think intelligence, per se, is highly overrated.\nSpeciXcally, I am unaware of any actual data that shows a cor-'], ground_truth='The author believes that intelligence is overrated because they are unaware of any actual data that shows a correlation between intelligence and company success.', evolution_type='simple', metadata=[{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 73, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [27]:
test_df = testset.to_pandas()

In [28]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,Why does the author believe that intelligence ...,[Criteria 7rst\nLots of people will tell you t...,The author believes that intelligence is overr...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,How important is it for a startup to focus on ...,"[developing a large market, as opposed to Xght...",Focusing on developing a large market is impor...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,What impact does the presence of a billion peo...,[billion people online now. That is a very lar...,The presence of a billion people online has le...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,What are the consequences of not raising enoug...,[Here’s why you shouldn’t do that:\nWhat are t...,Not raising enough money risks the survival of...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,How does a deal with a big company typically i...,[always be ready to have the conversation just...,A deal with a big company typically does not g...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,What industry is Washington DC known for in te...,[most interesting opportunity available — the ...,Washington DC is known for the politics industry.,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,What are some common layers of risk for a high...,[as if it’s an onion. Just like you peel an on...,Here are some common layers of risk for a high...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,How are high expectations beneficial in a high...,[There are a bunch of great things that you ge...,High expectations in a high-growth company can...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,How does the quality of output in a creative c...,[becomes irrelevant to determining the success...,Quality of output in a creative career does no...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,What are the criteria for evaluating candidate...,[How to hire the best people you've\never work...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [29]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [30]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [31]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [32]:
response_dataset[0]

{'question': 'Why does the author believe that intelligence is overrated?',
 'answer': "The author believes that intelligence is overrated because they think that hiring the smartest people does not guarantee a company's success.",
 'contexts': ['Especially in this industry.\nYou will read, hire the smartest people out there and your com-\npany’s success is all but guaranteed.\nI think intelligence, per se, is highly overrated.',
  'theory.\nThe paper then digs into possible correlations between intelli-\ngence as measured by such metrics as IQ, and creative output:',
  '[E]ven if a minimal level of intelligence is requisite for achieve-\nment, beyond a threshold of around IQ 120 (the actual amount\nvarying across Xelds), intellectual prowess becomes largely irrele-',
  'Now, clearly you don’t want to hire dumb people, and clearly\nyou’d like to work with smart people.\nBut let’s get speciXc.\nMost of the lore in our industry about the role of intelligence'],
 'ground_truth': 'The auth

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [33]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [34]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

No statements were generated from the answer.


In [35]:
results

{'faithfulness': 0.7481, 'answer_relevancy': 0.8522, 'context_recall': 0.5175, 'context_precision': 0.7690, 'answer_correctness': 0.4319}

In [36]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,Why does the author believe that intelligence ...,The author believes that intelligence is overr...,"[Especially in this industry.\nYou will read, ...",The author believes that intelligence is overr...,0.500000,1.000000,1.000000,0.416667,0.613280
1,How important is it for a startup to focus on ...,It is important for a startup to focus on deve...,"[competitor, be sure to take a step back and s...",Focusing on developing a large market is impor...,0.000000,0.938918,1.000000,1.000000,0.747455
2,What impact does the presence of a billion peo...,The presence of a billion people online is cau...,[billion people online now. That is a very lar...,The presence of a billion people online has le...,1.000000,0.886831,1.000000,1.000000,0.572266
3,What are the consequences of not raising enoug...,Not raising enough money risks the survival of...,[Here’s why you shouldn’t do that:\nWhat are t...,Not raising enough money risks the survival of...,1.000000,0.963161,0.500000,0.805556,0.533194
4,How does a deal with a big company typically i...,A deal with the right big company can have a h...,[There are times in the life of a startup when...,A deal with a big company typically does not g...,1.000000,0.966861,1.000000,1.000000,0.234217
5,What industry is Washington DC known for in te...,Politics.,[the city where all the action is happening.\n...,Washington DC is known for the politics industry.,1.000000,0.855998,1.000000,1.000000,0.960259
6,What are some common layers of risk for a high...,"Founder risk, technology risk, and product ris...",[What are the layers of risk for a high-tech\n...,Here are some common layers of risk for a high...,1.000000,0.993852,0.000000,1.000000,0.538804
7,How are high expectations beneficial in a high...,High expectations are beneficial in a high-gro...,[Third rule:\nIn a rapidly changing Held like ...,High expectations in a high-growth company can...,0.000000,0.992456,0.000000,1.000000,0.575004
8,How does the quality of output in a creative c...,The quality of output in a creative career rel...,[creator’s most distinguished work will appear...,Quality of output in a creative career does no...,1.000000,1.000000,0.000000,1.000000,0.225367
9,What are the criteria for evaluating candidate...,Criteria for evaluating candidates when hiring...,[How to hire the best people you've\never work...,nan,0.500000,1.000000,0.000000,0.000000,0.183508


## Task 2: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [37]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [39]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [40]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift fueding with?"})

In [41]:
print(response["answer"])

I'm sorry, but based on the context provided, I do not have information about any feud involving Taylor Swift.


In [42]:
response = retrieval_chain.invoke({"input": "Why are they fueding?"})

In [43]:
print(response["answer"])

There is no information provided in the context about any feud or conflict between individuals or groups.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [44]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [45]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [48]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics, raise_exceptions=False)

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/home/mpav/anaconda3/envs/llmops-w5/lib/python3.11/site-packages/ragas/executor.py", line 78, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/home/mpav/anaconda3/envs/llmops-w5/lib/python3.11/asyncio/tasks.py", line 615, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/home/mpav/anaconda3/envs/llmops-w5/lib/python3.11/site-packages/ragas/executor.py", line 37, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/home/mpav/anaconda3/envs/llmops-w5/lib/python3.11/site-packages/ragas/executor.py", line 111, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mpav/anaconda3/envs/llmops-w5/lib/python3.11/site-packages/ragas/metrics/base.py", line 125, in ascore
    raise e
  File "/home/mpav/anaconda3/envs/llmops-w5/lib/python3.11/site-package

In [49]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,Why does the author believe that intelligence ...,The author believes that intelligence is overr...,"[Especially in this industry.\nYou will read, ...",The author believes that intelligence is overr...,NaN,NaN,NaN,NaN,1.000000
1,How important is it for a startup to focus on ...,It is very important for a startup to focus on...,"[answer, in part because in the beginning of a...",Focusing on developing a large market is impor...,1.000000,1.000000,0.200000,0.528415,0.600000
2,What impact does the presence of a billion peo...,The presence of a billion people online has le...,"[Which makes total sense, amid the enormous ma...",The presence of a billion people online has le...,0.946296,1.000000,1.000000,0.564995,0.857143
3,What are the consequences of not raising enoug...,Not raising enough money risks the survival of...,[Here’s why you shouldn’t do that:\nWhat are t...,Not raising enough money risks the survival of...,0.989127,0.500000,1.000000,1.000000,0.963161
4,How does a deal with a big company typically i...,A deal with a big company typically does not l...,[There are times in the life of a startup when...,A deal with a big company typically does not g...,1.000000,0.810000,0.447298,1.000000,0.940896
5,What industry is Washington DC known for in te...,Washington DC is known for the politics industry.,[the city where all the action is happening.\n...,Washington DC is known for the politics industry.,1.000000,0.638889,0.243393,1.000000,0.925706
6,What are some common layers of risk for a high...,Some common layers of risk for a high-tech sta...,[What are the layers of risk for a high-tech\n...,Here are some common layers of risk for a high...,0.666667,1.000000,1.000000,1.000000,0.000000
7,How are high expectations beneficial in a high...,High expectations in a high-growth company can...,[Third rule:\nIn a rapidly changing Held like ...,High expectations in a high-growth company can...,0.950000,0.494258,0.000000,0.978908,1.000000
8,How does the quality of output in a creative c...,The relation between quantity and quality of o...,[creator’s most distinguished work will appear...,Quality of output in a creative career does no...,0.892857,0.779570,0.666667,0.949753,0.000000
9,What are the criteria for evaluating candidate...,"When hiring the best people, it is important t...",[with your team as you interview candidates fo...,nan,0.950000,0.223059,1.000000,0.950783,1.000000


## Task 3: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [50]:
results

{'faithfulness': 0.7481, 'answer_relevancy': 0.8522, 'context_recall': 0.5175, 'context_precision': 0.7690, 'answer_correctness': 0.4319}

And see how our advanced retrieval modified our chain!

In [52]:
advanced_retrieval_results

{'faithfulness': 0.6699, 'answer_relevancy': 0.7616, 'context_recall': 0.7756, 'context_precision': 0.7504, 'answer_correctness': 0.6951}

In [53]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.748148,0.669941,-0.078207
1,answer_relevancy,0.852193,0.761616,-0.090578
2,context_recall,0.517544,0.775565,0.258021
3,context_precision,0.769006,0.750441,-0.018565
4,answer_correctness,0.431944,0.695150,0.263205


## Task 4: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [54]:
# creating embedding model object instance

new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [55]:
# creating qdrant vector store stored in memory
vector_store = Qdrant.from_documents(
    documents,
    new_embeddings,
    location=":memory:",
    collection_name="PMarca Blogs - TE3 - MQR",
)

In [56]:
# create retriver

new_retriever = vector_store.as_retriever()

In [57]:
# advanced retriver using multi query retriver

new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [58]:


new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [59]:
# prepare data for evaluation dataset using new retrival chan

answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [60]:
# transform our lists in evaluation dataset

new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [62]:
# evaluate results of our new advanced retrival

new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics, raise_exceptions=False)

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/home/mpav/anaconda3/envs/llmops-w5/lib/python3.11/site-packages/ragas/executor.py", line 78, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/home/mpav/anaconda3/envs/llmops-w5/lib/python3.11/asyncio/tasks.py", line 615, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/home/mpav/anaconda3/envs/llmops-w5/lib/python3.11/site-packages/ragas/executor.py", line 37, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/home/mpav/anaconda3/envs/llmops-w5/lib/python3.11/site-packages/ragas/executor.py", line 111, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mpav/anaconda3/envs/llmops-w5/lib/python3.11/site-packages/ragas/metrics/base.py", line 125, in ascore
    raise e
  File "/home/mpav/anaconda3/envs/llmops-w5/lib/python3.11/site-package

In [63]:
# result of our our evaluation
# each score is the mean of individual scores of all the samples

new_advanced_retrieval_results

{'faithfulness': 0.6380, 'answer_relevancy': 0.8383, 'context_recall': 0.6390, 'context_precision': 0.6832, 'answer_correctness': 0.7167}

In [65]:
new_advanced_retrieval_results.items()

dict_items([('faithfulness', 0.6379657973587431), ('answer_relevancy', 0.8383408372712), ('context_recall', 0.6389906714815149), ('context_precision', 0.6832114840111884), ('answer_correctness', 0.7167423866934213)])

In [64]:
Ž# creating dataframe of all our retrivers

df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['ADA + MQR -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + MQR']
df_merged['Baseline -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + Baseline']

df_merged

,Metric,ADA + Baseline,ADA + MQR,TE3 + MQR,ADA + MQR -> TE3 + MQR,Baseline -> TE3 + MQR
0,faithfulness,0.748148,0.669941,0.637966,-0.031976,-0.110182
1,answer_relevancy,0.852193,0.761616,0.838341,0.076725,-0.013852
2,context_recall,0.517544,0.775565,0.638991,-0.136574,0.121447
3,context_precision,0.769006,0.750441,0.683211,-0.067230,-0.085794
4,answer_correctness,0.431944,0.695150,0.716742,0.021593,0.284798


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

because i was getting token rate limit a lot during evaluation, i woukldn't say that my results are representative.

Wiz reulsts were showing significant improvment using TE3 - text emb small regarding faithfullness, in my example te3 was better in answer relewancy and answer correctness.


## BONUS ACTIVITY: Using a Better Generator

Now that we've seen how much more effective a better Retrieval pipeline is, let's look at what impact a better(?) Generator is!

Adapt the above `TE3 + MQR` pipeline to use `GPT-4o` and compare the results below!

In [ ]:
### YOUR CODE HERE